In [4]:
from cv2 import blur
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack
import cv2
import random
from skimage import metrics
import os
import pandas as pd

img1 = cv2.imread('images\\img1_orig.jpg')
img2 = cv2.imread('images\\img2_orig.jpg')
img3 = cv2.imread('images\\img3_orig.jpg')
# bgr2rgb
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


# def mse(K, I):
#     m, n = K.shape[0], K.shape[1]
#     mse = 1 / m * n * np.sum((K - I) ** 2)
#     return mse
    
def MSE(img1, img2):
        squared_diff = (img1 -img2) ** 2
        summed = np.sum(squared_diff)
        num_pix = img1.shape[0] * img1.shape[1] #img1 and 2 should have same shape
        err = summed / num_pix
        return err

def mse(K, I):
    m, n = K.shape[0], K.shape[1]
    mse = 1 / m * n * np.sum((K - I) ** 2)
    return mse


def psnr(K, I):
    psnr = 10 * np.log10(255 ** 2 / MSE(K, I))
    return psnr


def ssim(K, I):
    return metrics.structural_similarity(K, I, channel_axis=-1)


def image_fidelity(K, I):
    return 1 - np.sum((K - I) ** 2) / np.sum(K * I)


def jpg_compress(img, quality):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    result, encimg = cv2.imencode('.jpg', img, encode_param)
    decimg = cv2.imdecode(encimg, 1)

    fig, axs = plt.subplots(1, 2, sharey=True)
    axs[0].imshow(img)
    axs[1].imshow(decimg)
    return decimg


def blur_img(img, maskSize):
    median = cv2.medianBlur(img, maskSize)
    return median


def noise_img(img, noise_type, samples_size):
    if noise_type == "speckle":
        gauss = np.random.normal(0, samples_size, img.size)
        gauss = gauss.reshape(img.shape[0], img.shape[1], img.shape[2]).astype("uint8")
        return img + img * gauss
    if noise_type == "gaussian":
        gauss = np.random.normal(0, samples_size, img.size)
        gauss = gauss.reshape(img.shape[0], img.shape[1], img.shape[2]).astype("uint8")
        return cv2.add(img, gauss)
    if noise_type == "exponential":
        exponential = np.random.exponential(samples_size, img.size)
        exponential = exponential.reshape(
            img.shape[0], img.shape[1], img.shape[2]
        ).astype("uint8")
        return cv2.add(img, exponential)
    if noise_type == "rayleigh":
        rayleigh = np.random.rayleigh(samples_size, img.size)
        rayleigh = rayleigh.reshape(img.shape[0], img.shape[1], img.shape[2]).astype(
            "uint8"
        )
        return cv2.add(img, rayleigh)
    if noise_type == "uniform":
        uniform = np.random.uniform(0, samples_size, img.size)
        uniform = uniform.reshape(img.shape[0], img.shape[1], img.shape[2]).astype(
            "uint8"
        )
        return cv2.add(img, uniform)

def create_blurred_images(img):
    dir = "C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\blur"
    f = open("C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\blur\\dane.txt", "w")
    os.chdir(dir)
    arr = [3, 5, 7, 9, 11, 15, 19, 25, 31, 41]
    for blur in arr:
        filename = f"blurred_maskSize-{blur}.jpg"
        distort = blur_img(img, blur)
        cv2.imwrite(filename, distort)
        data_to_write = "\"%s\",\"%s\",\"%s\",\"%s\",\"%s\"\n" % (filename,MSE(img, distort), psnr(img, distort), ssim(img, distort), image_fidelity(img, distort))
        f.write(data_to_write)

def create_compressed_images(img):
    dir = "C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\kompresjajpg"
    arr = [10,20,25,30,40,45,50,60,70,80]
    f = open("C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\kompresjajpg\\dane.txt", "w")
    os.chdir(dir)
    for quality in arr:
        filename = f"compressed_{quality}.jpg"
        distort = jpg_compress(img, quality)
        cv2.imwrite(filename, distort)
        data_to_write = "\"%s\",\"%s\",\"%s\",\"%s\",\"%s\"\n" % (filename,MSE(img, distort), psnr(img, distort), ssim(img, distort), image_fidelity(img, distort))
        f.write(data_to_write)
        quality += 10

def create_noise_images(img):
    dir = "C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\noise"
    f = open("C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\noise\\dane.txt", "w")
    os.chdir(dir)
    noise_types = ["gaussian", "rayleigh", "uniform", "exponential", "speckle"]
    noise_str = []
    for types in noise_types:
        if(types in ['gaussian', 'speckle']):
            noise_str = [0.4, 0.8]
        else:
            noise_str = [20, 40]
        
        for strength in noise_str:
            filename = f"noisy_{types}-{strength}.jpg"
            distort = noise_img(img, types, strength)
            cv2.imwrite(filename, distort)
            data_to_write = "\"%s\",\"%s\",\"%s\",\"%s\",\"%s\"\n" % (filename,MSE(img, distort), psnr(img, distort), ssim(img, distort), image_fidelity(img, distort))
            f.write(data_to_write)


def merge_miary_ocen():
    blur = "C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\blur\\dane.txt"
    kompresja = "C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\kompresjajpg\\dane.txt"
    noise = "C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\badania\\noise\\dane.txt"

    f = open("C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\merge.txt", "w")
    f.write("\"MSE\",\"PSNR\",\"SSIM\",\"FIDELITY\"\n")
    with open(noise) as noise_file:
        for line in noise_file:
            f.write(line)
    with open(blur) as blur_file:
        for line in blur_file:
            f.write(line)
    with open(kompresja) as kompresja_file:
        for line in kompresja_file:
            f.write(line)

def fakeDataFrame(dataframe):
    new_dataframe = dataframe.copy(deep=True)

    for col in new_dataframe:
        for i in range(new_dataframe[col].size):
            chance = random.random()
            if(chance < 0.2):
                if(new_dataframe[col][i] == 5):
                    new_dataframe[col][i] = new_dataframe[col][i]-1
                elif(new_dataframe[col][i] == 0):
                    new_dataframe[col][i] = new_dataframe[col][i]+1
                else:
                    if(random.random() > 0.5):
                        new_dataframe[col][i] = new_dataframe[col][i]-1
                    else:
                        new_dataframe[col][i] = new_dataframe[col][i]+1

    return new_dataframe

def makeManyRevievs(dataToChange):
    
    new_data1 = fakeDataFrame(dataToChange)
    new_data2 = fakeDataFrame(dataToChange)

    new_data1.columns = new_data1.columns.str.replace('0', '1')
    new_data2.columns = new_data2.columns.str.replace('0', '2')

    new_data = pd.concat([new_data1, new_data2], axis=1)
    new_data = pd.concat([dataToChange, new_data], axis=1)

    new_data.to_csv("C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10\\many_revievs.csv", index=False)
    return new_data



# create_noise_images(img3)
# create_blurred_images(img1)
# create_compressed_images(img2)
# merge_miary_ocen()

def make_pairs(df, norm):
    pairs_all_separately = []
    pairs_one_subject = []
    pairs_mean = []

    norms = ["MSE","PSNR","SSIM","FIDELITY"]

    for norm in norms:
        for i in range(df.shape[0]):
            user_values = df.iloc[:, :20]
            for j in range(user_values.shape[1]):
                norm_val = df[norm].values[i]
                pairs_all_separately.append([user_values.iloc[i, j], norm_val])

    for norm in norms:
        for i in range(df.shape[0]):
            user_values = df.iloc[:, :20]
            for j in range(user_values.shape[1]):
                norm_val = df[norm].values[i]
                pairs_all_separately.append([user_values.iloc[i, j], norm_val])

    for norm in norms:
        for i in range(data_array["MEAN"].size):
            norm_val = data_array[norm][i]
            arr.append([data_array["MEAN"][i],norm_val])

    arr = np.array(arr)

    return arr


os.chdir("C:\\Users\\Luberski\\Repos\\Systemy_Multimedialne\\Lab10")
data_array = pd.read_csv("jakosc_obrazow.csv").T
merge_array = pd.read_csv("merge.txt")
a = pd.read_csv("a.txt")
data_array = makeManyRevievs(data_array)
data_array["MEAN"] = data_array.mean(axis=1)
data_array['MSE'] = merge_array['MSE'].tolist()
data_array['PSNR'] = merge_array['PSNR'].tolist()
data_array['SSIM'] = merge_array['SSIM'].tolist()
data_array['FIDELITY'] = merge_array['FIDELITY'].tolist()


# arr = make_pairs(data_array)


# print(data_array)

# norms=np.arange(0,10)
# user_id_rep=100*id+10*rep+np.arange(0,10)

# miara_jakosci=np.arange(0,10)
# oceny_uzytk=100*id+10*5+np.arange(0,10)

# print(oceny_uzytk)
data_array.iloc[:, :48]

,badany1_0,badany2_0,badany3_0,badany4_0,badany5_0,badany6_0,badany7_0,badany8_0,badany9_0,badany10_0,...,badany8_2,badany9_2,badany12_2,badany11_2,badany12_2,badany13_2,badany14_2,badany15_2,badany16_2,MEAN
noisy_exponential-20,3,3,4,3,2,3,2,2,3,3,...,2,3,2,3,5,3,3,3,2,2.833333
noisy_exponential-40,1,3,3,2,1,2,1,1,3,2,...,0,3,1,2,1,2,2,2,1,1.750000
noisy_gaussian-0.4,3,4,4,3,3,3,2,3,4,4,...,3,5,4,2,5,2,3,4,3,3.333333
noisy_gaussian-0.8,2,2,2,1,1,1,1,1,2,1,...,2,2,1,1,3,2,1,1,2,1.625000
noisy_rayleigh-20,4,4,5,4,2,4,3,2,4,3,...,2,5,3,3,5,3,4,4,3,3.625000
noisy_rayleigh-40,3,3,3,2,1,2,1,2,2,2,...,2,1,2,2,4,2,2,3,2,2.166667
noisy_speckle-0.4,4,4,5,3,3,3,2,3,4,2,...,3,4,2,3,4,3,4,4,2,3.375000
noisy_speckle-0.8,1,3,3,1,1,2,1,1,2,1,...,1,1,0,1,1,2,1,1,1,1.416667
noisy_uniform-20,5,5,5,5,5,4,4,4,5,5,...,4,4,5,4,5,4,5,5,5,4.583333
noisy_uniform-40,4,5,5,4,2,4,2,3,4,4,...,3,4,4,3,4,3,5,4,5,3.812500
